**Célula de Texto: Solução para o Teste de Analista de Dados**

**Desenvolvido por: Ian Rodrigues**

Esta solução aborda os desafios propostos no teste de Analista de Dados, utilizando Python com as bibliotecas Pandas, openpyxl, mysql-connector-python e Graphviz para processar e analisar os dados do arquivo Excel "Extração_Teste_LeverPro.xlsx". O objetivo principal foi:

* Tratar os dados
* Modelar o banco de dados
* Verificar a integridade do balancete
* Gerar um resumo em formato JSON

**Principais Etapas e Resultados:**

1.  **Modelagem do Banco de Dados:**

    * Foi criado um **Diagrama Entidade-Relacionamento (DER)** no modelo estrela.
    * O DER representa as tabelas:
        * Empresa
        * Relatorio
        * CentroDeCusto
        * MovimentoContabil
    * Os relacionamentos entre as tabelas foram definidos.

2.  **Tratamento dos Dados:**

    * Os dados foram lidos e tratados utilizando **Pandas**.
    * As seguintes ações foram realizadas:
        * Limpeza e padronização de colunas
        * Conversões de tipo
        * Criação da coluna "Tipo" (Ativo/Passivo)

3.  **Geração de Scripts SQL:**

    * Scripts **SQL (DDL)** foram gerados para criar as tabelas no banco de dados.
    * Scripts de **inserção (INSERT)** foram gerados para popular as tabelas.

4.  **Verificação da Integridade:**

    * A integridade do balancete foi verificada calculando a soma de Ativo e Passivo.
    * O resultado foi apresentado em **formato JSON**, indicando se o balancete está íntegro ou não.

5.  **Geração de Resumo em JSON:**

    * Um arquivo **JSON** foi gerado, resumindo a integridade por empresa.
    * Este resumo facilita a análise e interpretação dos resultados.



In [1]:
# 📦 Instalações (apenas no Colab)
!pip install pandas openpyxl mysql-connector-python graphviz

# 📚 Imports
import pandas as pd
from google.colab import files
from graphviz import Digraph
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 39.4 MB/s eta 0:00:00


In [2]:
uploaded = files.upload()

Saving Extração_Teste_LeverPro.xlsx to Extração_Teste_LeverPro.xlsx


In [3]:
# Carregar planilhas
file = "Extração_Teste_LeverPro.xlsx"
xls = pd.ExcelFile(file)

empresa_df = xls.parse('Empresa')
movimento_df = xls.parse('Movimento Contábil')
relatorio_df = xls.parse('Relatório')
centro_custo_df = xls.parse('Centro de Custo')

# Padronizar colunas e limpar dados
for df in [empresa_df, movimento_df, relatorio_df, centro_custo_df]:
    df.columns = df.columns.str.strip()

relatorio_df["Relatório"] = relatorio_df["Relatório"].str.strip()
relatorio_df["Indicador Ativo / Inativo"] = relatorio_df["Indicador Ativo / Inativo"].str.strip()
centro_custo_df["Descrição"] = centro_custo_df["Descrição"].str.strip()
centro_custo_df["Indicador Ativo / Inativo"] = centro_custo_df["Indicador Ativo / Inativo"].str.strip()

movimento_df["Valor"] = pd.to_numeric(movimento_df["Valor"], errors="coerce")
movimento_df["Data"] = pd.to_datetime(movimento_df["Data"], errors="coerce")

# Identificar tipo (Ativo ou Passivo)
movimento_df["Tipo"] = movimento_df["Conta Contábil"].astype(str).str.startswith("1").map({True: "Ativo", False: "Passivo"})


In [4]:
create_scripts = """
CREATE TABLE Empresa (
    Codigo INT PRIMARY KEY,
    Empresa VARCHAR(100)
);

CREATE TABLE Relatorio (
    Codigo INT PRIMARY KEY,
    Relatorio VARCHAR(100),
    Indicador VARCHAR(10)
);

CREATE TABLE CentroDeCusto (
    Codigo INT PRIMARY KEY,
    Descricao VARCHAR(100),
    Indicador VARCHAR(10)
);

CREATE TABLE MovimentoContabil (
    Seq INT PRIMARY KEY,
    Data DATE,
    ContaContabil VARCHAR(30),
    Descricao VARCHAR(255),
    Valor DECIMAL(15, 2),
    Relatorio INT,
    CentroDeCusto INT,
    Empresa INT,
    Tipo VARCHAR(10),
    FOREIGN KEY (Relatorio) REFERENCES Relatorio(Codigo),
    FOREIGN KEY (CentroDeCusto) REFERENCES CentroDeCusto(Codigo),
    FOREIGN KEY (Empresa) REFERENCES Empresa(Codigo)
);
"""

print("-- CRIAÇÃO DE TABELAS --")
print(create_scripts)


-- CRIAÇÃO DE TABELAS --

CREATE TABLE Empresa (
    Codigo INT PRIMARY KEY,
    Empresa VARCHAR(100)
);

CREATE TABLE Relatorio (
    Codigo INT PRIMARY KEY,
    Relatorio VARCHAR(100),
    Indicador VARCHAR(10)
);

CREATE TABLE CentroDeCusto (
    Codigo INT PRIMARY KEY,
    Descricao VARCHAR(100),
    Indicador VARCHAR(10)
);

CREATE TABLE MovimentoContabil (
    Seq INT PRIMARY KEY,
    Data DATE,
    ContaContabil VARCHAR(30),
    Descricao VARCHAR(255),
    Valor DECIMAL(15, 2),
    Relatorio INT,
    CentroDeCusto INT,
    Empresa INT,
    Tipo VARCHAR(10),
    FOREIGN KEY (Relatorio) REFERENCES Relatorio(Codigo),
    FOREIGN KEY (CentroDeCusto) REFERENCES CentroDeCusto(Codigo),
    FOREIGN KEY (Empresa) REFERENCES Empresa(Codigo)
);



In [5]:
def gerar_inserts(df, tabela):
    inserts = []
    for _, row in df.iterrows():
        values = ', '.join(
            ["'{}'".format(str(x).replace("'", "")) if pd.notnull(x) else "NULL" for x in row.values]
        )
        insert = "INSERT INTO {} VALUES ({});".format(tabela, values)
        inserts.append(insert)
    return inserts

# Gerar os INSERTs para cada tabela
insert_sql = (
    gerar_inserts(empresa_df, "Empresa") +
    gerar_inserts(relatorio_df, "Relatorio") +
    gerar_inserts(centro_custo_df, "CentroDeCusto") +
    gerar_inserts(movimento_df, "MovimentoContabil")
)

# Exibir os primeiros INSERTs gerados
print("-- EXEMPLOS DE INSERTS --")
print("\n".join(insert_sql[:10]))


-- EXEMPLOS DE INSERTS --
INSERT INTO Empresa VALUES ('1', 'EMPRESA 1');
INSERT INTO Empresa VALUES ('2', 'EMPRESA 2');
INSERT INTO Empresa VALUES ('3', 'EMPRESA 3');
INSERT INTO Empresa VALUES ('4', 'EMPRESA 4');
INSERT INTO Empresa VALUES ('5', 'EMPRESA 5');
INSERT INTO Relatorio VALUES ('1', 'BALANCO PATRIMONIAL', 'S');
INSERT INTO Relatorio VALUES ('2', 'DRE', 'Sim');
INSERT INTO Relatorio VALUES ('3', 'INDICADORES', 'N');
INSERT INTO Relatorio VALUES ('4', 'FLUXO DE CAIXA', 'S');
INSERT INTO Relatorio VALUES ('5', 'DMPL', 'N');


In [6]:
soma_ativo = movimento_df[movimento_df["Tipo"] == "Ativo"]["Valor"].sum()
soma_passivo = movimento_df[movimento_df["Tipo"] == "Passivo"]["Valor"].sum()
integridade = soma_ativo + soma_passivo

status_integridade = "INTEGRIDADE OK" if integridade == 0 else "ERRO DE INTEGRIDADE"

print(f"""
==========================================
Resumo da Integridade Financeira
==========================================
Total Ativo   : {soma_ativo:,.2f}
Total Passivo : {soma_passivo:,.2f}
Soma Total    : {integridade:,.2f}
Status        : {status_integridade}
==========================================
""")



Resumo da Integridade Financeira
Total Ativo   : 5,583,794,523.00
Total Passivo : -263,667,908.13
Soma Total    : 5,320,126,614.87
Status        : ERRO DE INTEGRIDADE



In [7]:
json_resultado = (
    movimento_df.groupby(["Empresa", "Tipo"])
    .agg({"Valor": "sum"})
    .reset_index()
    .pivot(index="Empresa", columns="Tipo", values="Valor")
    .fillna(0)
    .assign(Soma_Total=lambda df: df["Ativo"] + df["Passivo"])
    .assign(Status=lambda df: df["Soma_Total"].apply(lambda x: "INTEGRIDADE OK" if x == 0 else "ERRO"))
)

json_str = json_resultado.reset_index().to_json(orient="records", indent=2)
with open("resultado_integridade.json", "w") as f:
    f.write(json_str)

files.download("resultado_integridade.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
dot = Digraph()

dot.node("F", "MovimentoContabil (Fato)")
dot.node("D1", "Empresa")
dot.node("D2", "Relatorio")
dot.node("D3", "CentroDeCusto")

dot.edges([("D1", "F"), ("D2", "F"), ("D3", "F")])
dot.render("modelo_estrela", format="png", cleanup=False)
dot.view()


'modelo_estrela.pdf'